<a href="https://colab.research.google.com/github/divyanshupatel17/safety-surveillance/blob/main/notebooks/02_fire_smoke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fire and Smoke Detection YOLOv8 Training Pipeline

Train YOLOv8 model to detect fire and smoke for safety surveillance.

Model saves to:
- Kaggle: /kaggle/working/models/fire_smoke/best.pt
- Colab: /content/drive/MyDrive/safety-models/fire_smoke/best.pt
- Local: ./models/fire_smoke/best.pt

## Step 1: Environment Setup and GPU Check

In [6]:
import os
import sys

try:
    IS_COLAB = 'google.colab' in str(get_ipython())
except:
    IS_COLAB = False

IS_KAGGLE = os.path.exists('/kaggle/input')

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    BASE_DIR = '/content/drive/MyDrive/safety-models'
    DATASET_PATH = '/content/fire-smoke-dataset'
    WORKING_DIR = '/content'
    print('Running on Colab')
    print('IMPORTANT: Go to Runtime > Change runtime type > Select GPU (T4 or better)')
elif IS_KAGGLE:
    BASE_DIR = '/kaggle/working/models'
    DATASET_PATH = '/kaggle/input/fire-and-smoke-dataset'
    WORKING_DIR = '/kaggle/working'
    print('Running on Kaggle')
    print('IMPORTANT: Go to Settings > Accelerator > Select GPU T4 x2')
else:
    BASE_DIR = './models'
    DATASET_PATH = './datasets/fire_smoke'
    WORKING_DIR = '.'
    print('Running locally')

MODELS_DIR = os.path.join(BASE_DIR, 'fire_smoke')
RESULTS_DIR = os.path.join(WORKING_DIR, 'results/fire_smoke')

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"\nModels: {MODELS_DIR}")
print(f"Dataset: {DATASET_PATH}")
print(f"Results: {RESULTS_DIR}")

Mounted at /content/drive
Running on Colab
IMPORTANT: Go to Runtime > Change runtime type > Select GPU (T4 or better)

Models: /content/drive/MyDrive/safety-models/fire_smoke
Dataset: /content/fire-smoke-dataset
Results: /content/results/fire_smoke


## Step 2: Install Packages

In [7]:
import subprocess

packages = ['ultralytics', 'opencv-python', 'pillow', 'pyyaml']
for pkg in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', pkg])

print('Installed')

Installed


## Step 3: Import Libraries and Verify GPU

In [8]:
import cv2
import numpy as np
import torch
import yaml
import json
import shutil
from datetime import datetime
from pathlib import Path
from ultralytics import YOLO
from PIL import Image

gpu_available = torch.cuda.is_available()
device = 0 if gpu_available else 'cpu'

print(f"PyTorch: {torch.__version__}")
print(f"GPU Available: {gpu_available}")

if gpu_available:
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Using Device: GPU (cuda:0)")
else:
    print("\nWARNING: No GPU detected!")
    print("Training will be VERY SLOW on CPU")
    print(f"\nCurrent Device: CPU")

PyTorch: 2.9.0+cu126
GPU Available: True
GPU Device: Tesla T4
GPU Memory: 15.83 GB
Using Device: GPU (cuda:0)


## Step 4: Download Dataset

In [9]:
if IS_COLAB:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'kagglehub'])
    import kagglehub

    print('Downloading Fire and Smoke dataset from Kaggle...')
    download_path = kagglehub.dataset_download('dataclusterlabs/fire-and-smoke-dataset')
    print(f"Downloaded to: {download_path}")

    DATASET_PATH = download_path
elif IS_KAGGLE:
    print('Using Kaggle input dataset')
else:
    print('Local mode: ensure dataset is in ./datasets/fire_smoke')

print(f"Dataset path: {DATASET_PATH}")

Using Colab cache for faster access to the 'fire-and-smoke-dataset' dataset.
Downloaded to: /kaggle/input/fire-and-smoke-dataset
Dataset path: /kaggle/input/fire-and-smoke-dataset


## Step 5: Prepare Dataset Structure

In [10]:
import glob
from sklearn.model_selection import train_test_split

dataset_exists = os.path.exists(DATASET_PATH)
print(f"Dataset exists: {dataset_exists}")

if dataset_exists:
    print('\nAnalyzing dataset structure...')

    all_images = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG']:
        all_images.extend(glob.glob(os.path.join(DATASET_PATH, '**', ext), recursive=True))

    print(f"Found {len(all_images)} images")

    if len(all_images) > 0:
        prepared_dir = os.path.join(WORKING_DIR, 'fire_smoke_prepared')
        os.makedirs(prepared_dir, exist_ok=True)

        for split in ['train', 'val', 'test']:
            os.makedirs(os.path.join(prepared_dir, 'images', split), exist_ok=True)
            os.makedirs(os.path.join(prepared_dir, 'labels', split), exist_ok=True)

        train_imgs, test_imgs = train_test_split(all_images, test_size=0.2, random_state=42)
        train_imgs, val_imgs = train_test_split(train_imgs, test_size=0.15, random_state=42)

        print(f"Split: Train={len(train_imgs)}, Val={len(val_imgs)}, Test={len(test_imgs)}")

        CLASS_NAMES = {0: 'fire', 1: 'smoke'}

        def copy_and_label(img_list, split):
            for idx, img_path in enumerate(img_list):
                img_name = os.path.basename(img_path)
                new_name = f"{split}_{idx:05d}{os.path.splitext(img_name)[1]}"

                shutil.copy2(img_path, os.path.join(prepared_dir, 'images', split, new_name))

                label_class = 0 if 'fire' in img_path.lower() else 1

                img = Image.open(img_path)
                w, h = img.size
                x_center, y_center = 0.5, 0.5
                width, height = 0.8, 0.8

                label_path = os.path.join(prepared_dir, 'labels', split,
                                         os.path.splitext(new_name)[0] + '.txt')
                with open(label_path, 'w') as f:
                    f.write(f"{label_class} {x_center} {y_center} {width} {height}\n")

        copy_and_label(train_imgs, 'train')
        copy_and_label(val_imgs, 'val')
        copy_and_label(test_imgs, 'test')

        DATASET_PATH = prepared_dir
        print(f"\nDataset prepared at: {DATASET_PATH}")
    else:
        print('No images found in dataset')
        CLASS_NAMES = {0: 'fire', 1: 'smoke'}
else:
    print('Dataset not found')
    CLASS_NAMES = {0: 'fire', 1: 'smoke'}

Dataset exists: True

Analyzing dataset structure...
Found 100 images
Split: Train=68, Val=12, Test=20

Dataset prepared at: /content/fire_smoke_prepared


## Step 6: Create Dataset Config

In [11]:
if os.path.exists(DATASET_PATH):
    yaml_path = os.path.join(DATASET_PATH, 'data.yaml')

    data_config = {
        'path': DATASET_PATH,
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(CLASS_NAMES),
        'names': list(CLASS_NAMES.values())
    }

    with open(yaml_path, 'w') as f:
        yaml.dump(data_config, f)

    print('Created data.yaml')
    print(f"Classes: {list(CLASS_NAMES.values())}")

    for split in ['train', 'val', 'test']:
        img_dir = os.path.join(DATASET_PATH, 'images', split)
        if os.path.exists(img_dir):
            count = len([f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))])
            print(f"{split}: {count} images")
else:
    print('Dataset not prepared')
    yaml_path = None

Created data.yaml
Classes: ['fire', 'smoke']
train: 68 images
val: 12 images
test: 20 images


## Step 7: Load Model

In [12]:
MODEL_SIZE = 's'

model = YOLO(f'yolov8{MODEL_SIZE}.pt')
params = sum(p.numel() for p in model.model.parameters()) / 1e6

print(f"Model: YOLOv8{MODEL_SIZE}")
print(f"Parameters: {params:.2f}M")

Model: YOLOv8s
Parameters: 11.17M


## Step 8: Train Model

In [13]:
if yaml_path is None:
    print('Cannot train without dataset')
elif not gpu_available:
    print('WARNING: No GPU - training will be slow')
    print('Reduce epochs and batch size')
else:
    print('Starting GPU training...')

    results = model.train(
        data=yaml_path,
        epochs=50,
        imgsz=640,
        batch=16,
        patience=15,
        device=0,
        project=os.path.join(WORKING_DIR, 'runs/detect'),
        name='fire_smoke_detector',
        exist_ok=True,
        optimizer='SGD',
        lr0=0.01,
        lrf=0.01,
        momentum=0.937,
        weight_decay=0.0005,
        warmup_epochs=3,
        box=7.5,
        cls=0.5,
        dfl=1.5,
        save=True,
        val=True,
        plots=True,
        workers=4,
        amp=True
    )
    print('Training complete')

Starting GPU training...
Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/fire_smoke_prepared/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fire_smoke_detector, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patie

## Step 9: Evaluate Model

In [14]:
best_path = os.path.join(WORKING_DIR, 'runs/detect/fire_smoke_detector/weights/best.pt')

if os.path.exists(best_path):
    eval_model = YOLO(best_path)
    val_results = eval_model.val(
        data=yaml_path,
        imgsz=640,
        batch=16,
        device=device
    )

    print('Performance Metrics:')
    if hasattr(val_results, 'box'):
        print(f"mAP50: {val_results.box.map50:.4f}")
        print(f"mAP50-95: {val_results.box.map:.4f}")
        print(f"Precision: {val_results.box.mp:.4f}")
        print(f"Recall: {val_results.box.mr:.4f}")

        if hasattr(val_results.box, 'ap_class_index'):
            print('Per-class mAP50:')
            for i, cls_id in enumerate(val_results.box.ap_class_index):
                name = CLASS_NAMES.get(int(cls_id), f"Class {cls_id}")
                print(f"  {name}: {val_results.box.ap50[i]:.4f}")
else:
    print('Best model not found')

Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2346.3±1608.3 MB/s, size: 1368.9 KB)
val: Scanning /content/fire_smoke_prepared/labels/val.cache... 12 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 12/12 5.0Mit/s 0.0s
val: /content/fire_smoke_prepared/images/val/val_00002.jpg: corrupt JPEG restored and saved
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 1.1s/it 1.1s
                   all         12         12      0.992          1      0.995       0.76
                  fire         12         12      0.992          1      0.995       0.76
Speed: 1.6ms preprocess, 13.8ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to /content/runs/detect/val
Performance Metrics:
mAP50: 0.9950
mAP50-95: 0.7599
Precision: 0.9918
Recall: 1.0000
Per-c

## Step 10: Save Model

In [15]:
source_path = os.path.join(WORKING_DIR, 'runs/detect/fire_smoke_detector/weights/best.pt')
final_path = os.path.join(MODELS_DIR, 'best.pt')

if os.path.exists(source_path):
    shutil.copy2(source_path, final_path)
    size_mb = os.path.getsize(final_path) / (1024 * 1024)

    print(f"Model saved: {final_path}")
    print(f"Size: {size_mb:.2f} MB")

    metadata = {
        'model_name': f'FireSmoke_YOLOv8{MODEL_SIZE}',
        'classes': CLASS_NAMES,
        'num_classes': len(CLASS_NAMES),
        'trained': datetime.now().isoformat(),
        'framework': 'YOLOv8',
        'image_size': 640,
        'path': final_path,
        'size_mb': round(size_mb, 2)
    }

    with open(os.path.join(MODELS_DIR, 'metadata.json'), 'w') as f:
        json.dump(metadata, f, indent=2)

    readme = f'''# Fire and Smoke Detection Model

Model: YOLOv8{MODEL_SIZE}
Classes: fire, smoke
Size: {size_mb:.2f} MB

## Usage

```python
from ultralytics import YOLO
model = YOLO('best.pt')
results = model.predict('image.jpg', conf=0.5)
results[0].show()
```
'''

    with open(os.path.join(MODELS_DIR, 'README.md'), 'w') as f:
        f.write(readme)

    print(f"Metadata: {os.path.join(MODELS_DIR, 'metadata.json')}")
else:
    print('Trained model not found')

Model saved: /content/drive/MyDrive/safety-models/fire_smoke/best.pt
Size: 21.47 MB
Metadata: /content/drive/MyDrive/safety-models/fire_smoke/metadata.json


## Step 11: Test Inference

In [16]:
if os.path.exists(final_path):
    test_model = YOLO(final_path)

    test_dir = os.path.join(DATASET_PATH, 'images', 'test')

    if os.path.exists(test_dir):
        samples = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.png'))][:5]

        if samples:
            print(f"Testing on {len(samples)} images\n")
            for img in samples:
                path = os.path.join(test_dir, img)
                results = test_model.predict(path, conf=0.5, save=True,
                                            project=RESULTS_DIR, name='test')

                if results and len(results[0].boxes) > 0:
                    print(f"{img}: {len(results[0].boxes)} detections")
                    for box in results[0].boxes:
                        cls = int(box.cls.item())
                        conf = box.conf.item()
                        name = CLASS_NAMES.get(cls, f'Class{cls}')
                        print(f"  {name}: {conf:.2%}")
                else:
                    print(f"{img}: no detections")
            print(f"\nResults saved: {RESULTS_DIR}/test/")
else:
    print('Model not found')

Testing on 5 images


image 1/1 /content/fire_smoke_prepared/images/test/test_00011.jpg: 640x480 (no detections), 44.9ms
Speed: 2.5ms preprocess, 44.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/results/fire_smoke/test
test_00011.jpg: no detections

image 1/1 /content/fire_smoke_prepared/images/test/test_00004.jpg: 640x384 (no detections), 43.4ms
Speed: 2.0ms preprocess, 43.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)
Results saved to /content/results/fire_smoke/test2
test_00004.jpg: no detections

image 1/1 /content/fire_smoke_prepared/images/test/test_00005.jpg: 640x480 1 fire, 12.9ms
Speed: 2.4ms preprocess, 12.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/results/fire_smoke/test3
test_00005.jpg: 1 detections
  fire: 51.36%

image 1/1 /content/fire_smoke_prepared/images/test/test_00008.jpg: 640x480 1 fire, 12.1ms
Speed: 2.8ms preprocess, 12.1ms inference, 1.2ms p

## Summary

Fire and smoke detection model trained and ready.

Download model from:
- Kaggle: /kaggle/working/models/fire_smoke/best.pt
- Colab: /content/drive/MyDrive/safety-models/fire_smoke/best.pt
- Local: ./models/fire_smoke/best.pt